В данном ноутбуке рассматриваются способы управлением процессов тренировки модели непосредственно во время тренировки - обратный вызов Keras:

ModelCheckpoint

EarlyStopping

ReduceLRonPlateau

простой вызов fit() позволяет нам "токнуть бумажный самолетик", но мы не можем управлять его полетом

обратный вызов Keras - объект класса, который помещается в аргументы метода fit и вызывается в разные моменты времени. Он может делать следующее:

Фиксация состояния модели в контрольных точках

Прерывание обучения, когда точность перестает расти

Корректировка параметров (шага оптимизатора)

Логирование

# EarlyStopping

In [ ]:
import keras.callbacks

# ОБЯЗАТЕЛЬНО ДЕЛАТЬ МАССИВ (ДАЖЕ ИЗ 1 ЭЛЕМЕНТА)!

stop = [callbacks.EarlyStopping(
    # наблюдение за точностью валидации
    monitor='val_accuracy',
    # прекращение тренировки, если точность не растет ДОЛЬШЕ чем одна эпоха
    patience=1,
)]

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['val_accuracy'])

model.fit(
    epochs=10,
    batch_size=32,
    # так как обратный вызов следит за проверочными данными, их необходимо передать
    validation_data=val_generator,
    # обратный вызов
    callbacks=stop
)

# ModelCheckpoint

In [ ]:
import keras.callbacks

checkpoints = [callbacks.ModelCheckpoint(
    # путь для сохранения
    filepath='saved_model.h5',
    # наблюдение за потерями на валидации
    monitor='val_loss',
    # сохранение только лучшей модели
    save_best_only=True 
)]

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['val_accuracy'])

model.fit(
    epochs=10,
    batch_size=32,
    validation_data=val_generator,
    # обратный вызов
    callbacks=checkpoints
)

# ReduceLROnPlateau

In [ ]:
from keras import callbacks

reduceLR = [(
    callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        # уменьшение скорости обучения в 10 раз
        factor=0.1,
        # уменьшение скорости, если потери не уменьшаются в течении 10 эпох
        patience=10
    )
)]

model.fit(
    epochs=10,
    batch_size=32,
    validation_data=val_generator,
    # обратный вызов
    callbacks=reduceLR
)

# Создание своего обратного вызова

In [ ]:
# любой обратный вызов по умолчанию имеет доступ к двум атрибутам:
# self.model
# self.validation_data
# их даже не надо указывать

# создадим обратный вызов, который сохраняет все активации слоев в виде numpy 
# после каждой эпохи

import keras
import numpy as np

# ОБЯЗАТЕЛЬНО такое наследование!
class ActivationLogger(keras.callbacks.Callback):
  # создание модели, которая будет возвращать активации слоев другой модели
  def create_model(self, model):
    self.model = model
    layers_outputs = [layer.output for layer in model.layers]
    self.activations_model = keras.models.Model(model.input, layers_outputs)

  def on_epoch_end(self, epoch, logs=None):
    # self.validation_data - аргумент из keras.callbacks.Callback
    if self.validation_data is None:
      raise RuntimeError("Please provide validation data!")
    # получение первого образца из проверочных данных
    validation_sample = self.validation_data[0][0:1]
    activations = self.activations_model.predict(validation_sample)
    # сохранение массива на диск
    with open(f"activations_on_epoch {epoch}.npz", 'wb') as f:
      np.savez(f, activations)
